In [1]:
import ROOT
import json
import sys                                                                                                                                                                  
import os
import warnings
from datetime import time

myDir = '/eos/user/d/dalfonso/SWAN_projects/Hrare/JULY_exp'
sys.path.insert(0, myDir)

from utilsAna import pickTRG,getMesonFromJson,getMVAFromJson
from utilsAna import loadUserCode, loadCorrectionSet
from utilsAna import computeWeigths
#from utilsHrare import getMClist, getDATAlist, getSkims                                                                                                                                                                          
#from utilsAna import getDATAlist                                                                                                                                                                                                 
#from utilsAna import listDir                                                                                                                                                                                                     
from utilsAna import listLocalFiles
from utilsAna import BuildDictMgamma, SwitchSample
#from dictionary import ReadDictionary                                                                                                                                                                                            
from utilsAna import readDataQuality

doINTERACTIVE=False
doLocalCluster=False

AF="cern-dask"

doRemoteAccess=True

###-----------------------------------------                                                                                                                                                                                      
###-----------------------------------------                   

def init():
    loadUserCode()
    loadCorrectionSet(2018)
#    readDataQuality(2018)
    ROOT.gInterpreter.ProcessLine('#include "/eos/home-d/dalfonso/SWAN_projects/Hrare/JULY_exp/myLibrary.h"')
#    ROOT.gInterpreter.ProcessLine('#include "/eos/home-d/dalfonso/SWAN_projects/Hrare/JULY_exp/config/functions.h"')

###-----------------------------------------                                                                                                                                 
###-----------------------------------------                                                                                                                                 

if doINTERACTIVE:
#    ROOT.ROOT.EnableImplicitMT(64)
#    RDataFrame = ROOT.RDataFrame
    RunGraphs = ROOT.RDF.RunGraphs
    init()
else:
    
    if doLocalCluster:
        from utilsDask import create_local_connection # for MIT/Purdue
    elif AF=="subMIT" or AF=="Purdue":
        from utilsDask import create_remote_connection                                                                                                                                                                                             
        RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame # for MIT-purdue
    elif AF=="cern-spark":
        import pyspark
        RDataFrame = ROOT.RDF.Experimental.Distributed.Spark.RDataFrame # for CERN
    elif AF=="cern-dask":
        RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame # for CERN
    RunGraphs = ROOT.RDF.Experimental.Distributed.RunGraphs
        
    #    load User Library 
    ROOT.RDF.Experimental.Distributed.initialize(init)



Welcome to JupyROOT 6.30/04
loadUserCode()
loadCorrectionSet()


In [2]:
ROOT.CountCharacters("Hello, let me try again today")

29

In [ ]:
import ROOT
import os
os.environ["X509_USER_PROXY"] = "/eos/user/d/dalfonso/x509up_u7592"
print(os) 
myFile = ROOT.TFile.Open("root://xrootd.cmsaf.mit.edu//store/user/paus/nanohr/D02/VBF_HToPhiGamma_M125_TuneCP5_PSWeights_13TeV_powheg_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM/CAAC1BB3-38EA-824C-9E9E-B4A5E2A34CB4.root")
myFile.ls()

In [ ]:
import sys
from utilsAna import * 

###-----------------------------------------                                                                                                                                                                                      
###-----------------------------------------                                                                                                                                                                                      

def set_env():
    os.environ["X509_USER_PROXY"] = "/eos/user/d/dalfonso/x509up_u7592"

with open("./config/selection.json") as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()

with open("./config/trigger.json") as trgJsonFile:
    trgObject = json.load(trgJsonFile)
    trgJsonFile.close()

#with open("./config/qualityData.json") as qualityJsonFile:
#    qualObject = json.load(qualityJsonFile)
#    qualityJsonFile.close()

#JSON = qualObject['JSON']

BARRELphotons = jsonObject['BARRELphotons']
ENDCAPphotons = jsonObject['ENDCAPphotons']
#TRIGGERS = trgObject['triggers']                                                                                                                                                                                                 
mesons = jsonObject['mesons']
GOODJETS = jsonObject['GOODJETS']

year = 2018

GOODphotons = "({0} or {1}) and Photon_pt>38 and Photon_electronVeto and Photon_mvaID_WP80 and abs(Photon_eta)<2.1".format(BARRELphotons,ENDCAPphotons) #90-80                                                                    
GOODPHI = "{}".format(getMesonFromJson(mesons, "isZinv", "isPhiCat"))
GOODRHO = "{}".format(getMesonFromJson(mesons, "isZinv", "isRhoCat"))
GOODK0STAR = "{}".format(getMesonFromJson(mesons, "isZinv", "isK0StarCat"))

#TRIGGER=pickTRG(TRIGGERS,year,"NULL",True,False,False,False,False)                                                                                                                                                               
TRIGGER="HLT_Photon35_TwoProngs35"

print(GOODphotons)
print(GOODPHI)
print(TRIGGER)

lumis={
    '12016': 19.52, #APV #(B-F for 2016 pre)                                                                                                                                                                                      
    '22016': 16.80, #postVFP                                                                                                                                                                                                      
    '2016': 35.9,
    '2017': 41.5,
    '12017': 7.7, #(F for 2017)                                                                                                                                                                                                   
    '2018': 59.70,
    '12018': 39.54,
    'all': 86.92,      #19.52 + 7.7 + 59.70                                                                                                                                                                                       
}


###-----------------------------------------
###-----------------------------------------

histos = []
evtcounts = []

def analysis(files,year,mc,sumW):

    now = time()
    print('==> start: ',now)

    if doINTERACTIVE:
        dfINI = RDataFrame("Events", files)

    else:
        #files_per_partition = nfiles / npartitions                                                                                                                                                                               
        nClusters=64 # this is the total number of cores                                                                                                                                                                          
        NPARTITIONS=10 # this the cluster each graph get 64 / (2 signal + 5 MBbkg + 3 data)                                                                                                                                       
        if doLocalCluster:
            connection = create_local_connection(NPARTITIONS)
#            connection = create_local_connection(nClusters)                                                                                                                                                                      
            print(connection)
            # The `npartitions` optional argument tells the RDataFrame how many tasks are desired                                                                                                                                 
            dfINI = RDataFrame("Events", files, daskclient=connection, npartitions=NPARTITIONS)
        elif AF=="cern-spark":
            dfINI = RDataFrame("Events", files, sparkcontext=sc, npartitions=NPARTITIONS) # at CERN
            sc.addPyFile("/eos/home-d/dalfonso/SWAN_projects/Hrare/JULY_exp/utilsAna.py")
            #sc.addPyFile("/eos/home-d/dalfonso/SWAN_projects/Hrare/JULY_exp/myLibrary.h")
            print(sc.environment)
        elif AF=="cern-dask":
            from dask.distributed import Client
            client = Client("tls://10.100.7.153:30834")
            dfINI = RDataFrame("Events", files, daskclient=client, npartitions=NPARTITIONS)
            client.run(set_env)
            client.upload_file("/eos/home-d/dalfonso/SWAN_projects/Hrare/JULY_exp/utilsAna.py")
            print(client)
        elif AF=="subMIT" or AF=="Purdue":
            client = create_remote_connection()
            print(client)
            dfINI = RDataFrame("Events", files, daskclient=client, npartitions=NPARTITIONS)

    lumi = 1.
    weight = "{0}".format(1.)
    if mc>=0: weight = "{0}*genWeight*{1}".format(lumi,sumW)
    if year==2018: lumiIntegrated = lumis['2018']
    print(weight)
        
    isData = "false"
    if mc<0: isData = "true"

    print("DOING all the defines")
    df = (dfINI
          .Filter("nPhoton>0 and PV_npvsGood>0","photon from nano >0 and PV_npvsGood > 0")
          .Define("w","{}".format(weight))
          .Define("isData","{}".format(isData))
#          .Define("applyJson","{}".format(JSON)).Filter("applyJson","pass JSON")
          .Define("lumiIntegrated","{}".format(lumiIntegrated))
          #          .Define("triggerAna","{}".format(TRIGGER)) # note some rootfile doens't have this  trigger, on scratch no problem since we have already selected them                                                                                                                                                                            
#          .Filter("triggerAna>0", "pass triggers")                                                                                                                                                                                                                                                                                                   
          .Define("goodPhotons", "{}".format(GOODphotons))
          .Define("nGoodPhotons","Sum(goodPhotons)*1.0f")
          .Filter("Sum(goodPhotons)>0", "At least one good Photon")
          .Define("goodPhotons_pt", "Photon_pt[goodPhotons]")
          .Define("goodPhotons_eta", "Photon_eta[goodPhotons]")
          .Define("goodPhotons_phi", "Photon_phi[goodPhotons]")
          .Filter("nphi>0").Define("goodMeson","({}".format(GOODPHI)+")")
          .Filter("Sum(goodMeson)>0", "one good Phi (ptPhi, validfit, ptTracks)")
          .Define("goodMeson_pt", "phi_kin_pt[goodMeson]")
          .Define("goodMeson_eta", "phi_kin_eta[goodMeson]")
          .Define("goodMeson_phi", "phi_kin_phi[goodMeson]")
          .Define("goodMeson_mass", "phi_kin_mass[goodMeson]")
          #                                                                                                                                                                                                                                                                                                                                           
          .Define("goodMeson_iso", "phi_iso[goodMeson]")
          .Define("goodMeson_DR","DeltaR(phi_trk1_eta[goodMeson],phi_trk2_eta[goodMeson],phi_trk1_phi[goodMeson],phi_trk2_phi[goodMeson])")
          .Define("goodMeson_trk1_pt", "phi_trk1_pt[goodMeson]")
          .Define("goodMeson_trk2_pt", "phi_trk2_pt[goodMeson]")
          .Define("wrongMeson","({}".format(GOODRHO)+")")
          .Define("wrongMeson_pt","Sum(wrongMeson) > 0 ? rho_kin_pt[wrongMeson]: ROOT::VecOps::RVec<float>(0.f)")
          .Define("wrongMeson2","({}".format(GOODK0STAR)+")")
          .Define("wrongMeson2_pt","Sum(wrongMeson2) > 0 ? K0Star_kin_pt[wrongMeson2]: ROOT::VecOps::RVec<float>(0.f)")
          #                                                                                                                                                                                                                                                                                                                                           
          .Define("myInt","ROOT.CountCharacters('Hello, let me try again today.')")
#          .Define("index_pair","ROOT.HiggsCandFromRECO(goodMeson_pt, goodMeson_eta, goodMeson_phi, goodMeson_mass, goodMeson_trk1_pt, goodMeson_trk2_pt, wrongMeson_pt, wrongMeson2_pt, goodPhotons_pt, goodPhotons_eta, goodPhotons_phi)").Filter("index_pair[0]!= -1", "at least a good meson candidate")
#          .Define("meson_pt", "(index_pair[0]!= -1) ? goodMeson_pt[index_pair[0]]: 0.f")
#          .Define("photon_pt", "(index_pair[1]!= -1) ? goodPhotons_pt[index_pair[1]]: 0.f")
#          .Define("HCandMass", "compute_HiggsVars_var(meson_pt,goodMeson_eta[index_pair[0]],goodMeson_phi[index_pair[0]],goodMeson_mass[index_pair[0]],photon_pt,goodPhotons_eta[index_pair[1]],goodPhotons_phi[index_pair[1]],0)")
#          .Define("HCandPT",   "compute_HiggsVars_var(meson_pt,goodMeson_eta[index_pair[0]],goodMeson_phi[index_pair[0]],goodMeson_mass[index_pair[0]],photon_pt,goodPhotons_eta[index_pair[1]],goodPhotons_phi[index_pair[1]],1)")
          #                                                                                                                                                                                                                                                                                                                                           
          )

#    df = callVary(df)                                                                                                                                                                                                                                                                                                                                
#    df = callVariation(df) # only works for photon_pt (both interactive and distr)                                                                                                                                                                                                                                                                   

    evtcounts.append(df.Count())


    if True:
        print("writing plots")
        hists = {
            "goodPhotons_pt":  {"name":"goodPhotons_pt","title":"Photon PT; pt_{#gamma} (GeV);N_{Events}","bin":200,"xmin":0,"xmax":200},                                                                                                                                                                                                                      
#            "photon_pt":  {"name":"photon_pt","title":"Photon PT; pt_{#gamma} (GeV);N_{Events}","bin":200,"xmin":0,"xmax":200},                                                                                                                                                                                                                      
#            "HCandMass":  {"name":"HCandMass","title":"H mass;m_{k^{+}k^{-}#gamma} (GeV);N_{Events}","bin":70,"xmin":100,"xmax":170},
        }
    
        for h in hists:

            # 1D is for nom only                                                                                                                                                                                                                                                                                                                      
            model1d = (hists[h]["name"]+"_"+str(year)+"_"+str(mc), hists[h]["title"], hists[h]["bin"], hists[h]["xmin"], hists[h]["xmax"])
            h1d = df.Histo1D(model1d, hists[h]["name"], "w")
            histos.append(h1d)
            print("h1d append")

    if False:

        branchList = ROOT.vector('string')()
        for branchName in [
                "goodPhotons_pt",
#                "HCandMass",
        ]:
            branchList.push_back(branchName)

        outputFile = "DASKlogs/snapshotOUTname.root"
        print(outputFile)
        snapshotOptions = ROOT.RDF.RSnapshotOptions()
        snapshotOptions.fCompressionAlgorithm = ROOT.kLZ4
        snapshot_tdf = df.Snapshot("events", outputFile, branchList, snapshotOptions)
        print("snapshot_tdf DONE")
        print(outputFile)

    now = time()
    print('==> ends: ',now)

def loopRemoteDataset():

#    thisdict = BuildDictMgamma()

    year=2018
#    mc = [1017,1010,10,11,12,13,14]                                                                                                                                                                                                                                                                                                                  
    mc = [1017]
#    mc = []                                                                                                                                                                                                                                                                                                                                          

    for sampleNOW in mc:
#        files = SwitchSample(thisdict,sampleNOW)[0]
#        xsec = SwitchSample(thisdict,sampleNOW)[1]
#        print(files)                                                                                                                                                                                                                                                                                                                                 
#        files = ["root://xrootd.cmsaf.mit.edu//store/user/paus/nanohr/D02/VBF_HToPhiGamma_M125_TuneCP5_PSWeights_13TeV_powheg_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM/CAAC1BB3-38EA-824C-9E9E-B4A5E2A34CB4.root"]
#        files = "root://cms-xrd-global.cern.ch:1094//store/user/paus/nanohr/D02/VBF_HToPhiGamma_M125_TuneCP5_PSWeights_13TeV_powheg_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM/00B7EB0B-2697-F241-BE5F-1735C9656C30.root"
#        files = "root://eosuser.cern.ch//eos/user/d/dalfonso/Hrare/D02/GJets_HT-40To100_TuneCP5_13TeV-madgraphMLM-pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v2+MINIAODSIM/A50B4DC6-222A-8349-AFBD-FED3C3DBB001.root"
        files = listLocalFiles("/eos/user/d/dalfonso/Hrare/D02/GJets_HT-40To100_TuneCP5_13TeV-madgraphMLM-pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v2+MINIAODSIM/")
#        files = "/eos/user/d/dalfonso/CAAC1BB3-38EA-824C-9E9E-B4A5E2A34CB4.root"
#        print(files)
        xsec = 23804.2
#        print('outside the function: ',len(files))
#        rdf = ROOT.RDataFrame("Runs", files) # make sure this is not the distributed                                                                                                                                                                                                                                                                  
#        sumW = computeWeigths(rdf,xsec)
        sumW = 1.
        analysis(files,year,sampleNOW,sumW)

#    data = [ -62,-63,-64]                                                                                                                                                                                                                                                                                                                            
#    data = [ -63 ]                                                                                                                                                                                                                                                                                                                                   
    data = [ ]

    for datasetNumber in data:
        files = SwitchSample(thisdict,datasetNumber)[0]
#        print(files)                                                                                                                                                                                                                                                                                                                                 
        print(len(files))
        analysis(files,year,datasetNumber,1.)


# In a Python script the Dask client needs to be initalized in a context                                                                                                                                                                                                                                                                              
# Jupyter notebooks / Python session don't need this                                                                                                                                                                                                                                                                                                  
if __name__ == "__main__":

    now = time()
    print('==> very beginning: ',now)

    loopRemoteDataset()

    if True:

        now = time()
        print('==> start runGraph: ',now)
        RunGraphs(evtcounts);

        now = time()
        print('==> done runGraph: ',now)

        if doINTERACTIVE: outputFileHisto = "/eos/home-d/dalfonso/SWAN_projects/Hrare/JULY_exp/DASKlogs/histoOUTname_test_interactive.root"
        else:
            if doLocalCluster: outputFileHisto = "/eos/home-d/dalfonso/SWAN_projects/Hrare/JULY_exp/DASKlogs/histoOUTname_test_LocalCluster.root"
            else: outputFileHisto = "/eos/home-d/dalfonso/SWAN_projects/Hrare/JULY_exp/DASKlogs/histoOUTname_test_RemoteCluster.root"
        myfile = ROOT.TFile(outputFileHisto,"RECREATE")
        myfile.ls()

        print(histos)
        for h in histos:
            h.Write()
        myfile.Close()
        myfile.Write()

        now = time()
        print('==> histo done: ',now)
    
    
    

((Photon_pt>30 and Photon_isScEtaEB and Photon_mvaID_WP90) or (Photon_pt>30 and Photon_isScEtaEE and Photon_mvaID_WP80)) and Photon_pt>38 and Photon_electronVeto and Photon_mvaID_WP80 and abs(Photon_eta)<2.1
(abs(phi_kin_mass-1.02)<3*0.004 && phi_kin_pt>38 && phi_trk1_pt>15 && phi_trk2_pt>15 && phi_kin_valid && phi_iso > 0.9 && abs(phi_kin_eta)<2.1 && abs(phi_bestVtx_Z)<24 && phi_bestVtx_idx<5)
HLT_Photon35_TwoProngs35
==> very beginning:  00:00:00
/eos/user/d/dalfonso/Hrare/D02/GJets_HT-40To100_TuneCP5_13TeV-madgraphMLM-pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v2+MINIAODSIM/
702
==> start:  00:00:00
<Client: 'tls://10.100.7.153:30834' processes=0 threads=0, memory=0 B>
1.0*genWeight*1.0
DOING all the defines
writing plots
h1d append
==> ends:  00:00:00
==> start runGraph:  00:00:00


In [3]:
import uproot
f = uproot.open('/eos/home-d/dalfonso/SWAN_projects/Hrare/JULY_exp/DASKlogs/histoOUTname_test_RemoteCluster.root')
f['goodPhotons_pt_2018_1017'].to_hist()

Hist(Regular(200, 0, 200, name='xaxis', label=' pt_{#gamma} (GeV)'), storage=Double()) # Sum: 1250.0

In [ ]:
from dask.distributed import Client

client = Client("tls://10.100.7.153:30834")
client